In [2]:
## First, check to see if lightning is installed, if not, install it.
import pip
try:
  __import__("lightning")
except ImportError:
  pip.main(['install', "lightning"])  

import torch ## torch let's us create tensors and also provides helper functions
import torch.nn as nn ## torch.nn gives us nn.Module(), nn.Embedding() and nn.Linear()
import torch.nn.functional as F # This gives us the softmax() and argmax()
from torch.optim import Adam ## We will use the Adam optimizer, which is, essentially, 
                             ## a slightly less stochastic version of stochastic gradient descent.
from torch.utils.data import TensorDataset, DataLoader ## We'll store our data in DataLoaders

import lightning as L ## Lightning makes it easier to write, optimize and scale our code

In [3]:
class PositionEncoding(nn.Module):
    
    def __init__(self, d_model=2, max_len=6):
        
        super().__init__()
        
        pe = torch.zeros(max_len, d_model)
        
        position = torch.arange(start=0, end=max_len, step=1).float().unsqueeze(1)
        embedding_index = torch.arange(start=0, end=d_model, step=2).float()
        
        div_term = 1/torch.tensor(10000.0)**(embedding_index / d_model)
        

        pe[:, 0::2] = torch.sin(position * div_term) 
        pe[:, 1::2] = torch.cos(position * div_term) 
        
        self.register_buffer('pe', pe) 

        
    def forward(self, word_embeddings):
        
        return word_embeddings + self.pe[:word_embeddings.size(0), :] 

In [4]:
class Attention(nn.Module): 
    
    def __init__(self, d_model=2):
        
        super().__init__()
        
        self.W_q = nn.Linear(in_features=d_model, out_features=d_model, bias=False)
        self.W_k = nn.Linear(in_features=d_model, out_features=d_model, bias=False)
        self.W_v = nn.Linear(in_features=d_model, out_features=d_model, bias=False)
        
        self.row_dim = 0
        self.col_dim = 1

        
    def forward(self, encodings_for_q, encodings_for_k, encodings_for_v, mask=None):

        q = self.W_q(encodings_for_q)
        k = self.W_k(encodings_for_k)
        v = self.W_v(encodings_for_v)

        sims = torch.matmul(q, k.transpose(dim0=self.row_dim, dim1=self.col_dim))

        scaled_sims = sims / torch.tensor(k.size(self.col_dim)**0.5)

        if mask is not None:
            mask = mask.to(scaled_sims.device)  # Move mask to the same device as scaled_sims
            scaled_sims = scaled_sims.masked_fill(mask=mask, value=-1e9)
        
        attention_percents = F.softmax(scaled_sims, dim=self.col_dim)
        attention_scores = torch.matmul(attention_percents, v)
        
        return attention_scores

In [5]:
class DecoderOnlyTransformer(L.LightningModule):
    
    def __init__(self, num_tokens=4, d_model=2, max_len=6):
        
        super().__init__()
        
        L.seed_everything(seed=42)
        
        self.we = nn.Embedding(num_embeddings=num_tokens, 
                               embedding_dim=d_model)     
        self.pe = PositionEncoding(d_model=d_model, 
                                   max_len=max_len)
        self.self_attention = Attention(d_model=d_model)
        self.fc_layer = nn.Linear(in_features=d_model, out_features=num_tokens)
        
        self.loss = nn.CrossEntropyLoss()
        
        
    def forward(self, token_ids):
                
        word_embeddings = self.we(token_ids)        
        position_encoded = self.pe(word_embeddings)
        
        mask = torch.tril(torch.ones((token_ids.size(dim=0), token_ids.size(dim=0))))
        mask = mask == 0
        
        self_attention_values = self.self_attention(position_encoded, 
                                                    position_encoded, 
                                                    position_encoded, 
                                                    mask=mask)
                
        residual_connection_values = position_encoded + self_attention_values        
        fc_layer_output = self.fc_layer(residual_connection_values)
        
        return fc_layer_output
    
  
    def configure_optimizers(self): 
        return Adam(self.parameters(), lr=0.1)
    
    
    def training_step(self, batch, batch_idx): 
        input_tokens, labels = batch # collect input
        output = self.forward(input_tokens[0])
        loss = self.loss(output, labels[0])
                    
        return loss

In [6]:
## first, we create a dictionary that maps vocabulary tokens to id numbers...
token_to_id = {'what' : 0,
               'is' : 1,
               'apple' : 2,
               'healthy': 3,
               '<EOS>' : 4, ## <EOS> = end of sequence
              }
id_to_token = dict(map(reversed, token_to_id.items()))
inputs = torch.tensor([[token_to_id["what"], ## input #1: what is statquest <EOS> awesome
                        token_to_id["is"], 
                        token_to_id["apple"], 
                        token_to_id["<EOS>"],
                        token_to_id["healthy"]], 
                       
                       [token_to_id["apple"], # input #2: statquest is what <EOS> awesome
                        token_to_id["is"], 
                        token_to_id["what"], 
                        token_to_id["<EOS>"], 
                        token_to_id["healthy"]]])

## NOTE: Because we are using a Decoder-Only Transformer the outputs, or
##       the predictions, are the input questions (minus the first word) followed by 
##       <EOS> awesome <EOS>.  The first <EOS> means we're done processing the input question
##       and the second <EOS> means we are done generating the output.
##       See the illustration above for more details.
labels = torch.tensor([[token_to_id["is"], 
                        token_to_id["apple"], 
                        token_to_id["<EOS>"], 
                        token_to_id["healthy"], 
                        token_to_id["<EOS>"]],  
                       
                       [token_to_id["is"], 
                        token_to_id["what"], 
                        token_to_id["<EOS>"], 
                        token_to_id["healthy"], 
                        token_to_id["<EOS>"]]])

## Now let's package everything up into a DataLoader...
dataset = TensorDataset(inputs, labels) 
dataloader = DataLoader(dataset)

In [7]:
print(inputs)

tensor([[0, 1, 2, 4, 3],
        [2, 1, 0, 4, 3]])


In [8]:
print(labels)

tensor([[1, 2, 4, 3, 4],
        [1, 0, 4, 3, 4]])


In [9]:
model = DecoderOnlyTransformer(num_tokens=len(token_to_id), d_model=2, max_len=6)

## Now create the input for the transformer...
model_input = torch.tensor([token_to_id["what"], 
                            token_to_id["is"], 
                            token_to_id["apple"], 
                            token_to_id["<EOS>"]])
input_length = model_input.size(dim=0)

Seed set to 42


In [10]:
trainer = L.Trainer(max_epochs=30)
trainer.fit(model, train_dataloaders=dataloader)

You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
C:\Users\Ltx\anaconda3\envs\dl_gpu\Lib\site-packages\lightning\pytorch\trainer\connectors\logger_connector\logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
You are using a CUDA device ('NVIDIA GeForce RTX 3050 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precis

Training: |                                                                                                   …

`Trainer.fit` stopped: `max_epochs=30` reached.


In [11]:
print("Model input shape before embedding:", model_input.shape)
output = model(model_input)
print("Output shape:", output.shape)


Model input shape before embedding: torch.Size([4])
Output shape: torch.Size([4, 5])


In [32]:
model_input = torch.tensor([token_to_id["apple"], 
                            token_to_id["what"], 
                            token_to_id["is"], 
                            token_to_id["<EOS>"]])
input_length = model_input.size(dim=0)

predictions = model(model_input) 
predicted_id = torch.tensor([torch.argmax(predictions[-1,:])])
predicted_ids = predicted_id
max_length = 6
for i in range(input_length, max_length):
    if (predicted_id == token_to_id["<EOS>"]): # if the prediction is <EOS>, then we are done
        break
    
    model_input = torch.cat((model_input, predicted_id))
    
    predictions = model(model_input) 
    predicted_id = torch.tensor([torch.argmax(predictions[-1,:])])
    predicted_ids = torch.cat((predicted_ids, predicted_id))
        
print("Predicted Tokens:\n") 
for id in predicted_ids: 
    print("\t", id_to_token[id.item()])

Predicted Tokens:

	 healthy
	 <EOS>


In [25]:
!pip install transformers

   ---------------------------------------- 0.0/10.2 MB ? eta -:--:--
   ----------------------- ---------------- 6.0/10.2 MB 33.5 MB/s eta 0:00:01
   ---------------------------------------- 10.2/10.2 MB 35.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ---------------------------------------- 2.4/2.4 MB 46.2 MB/s eta 0:00:00


In [15]:
import torch
from torch.utils.data import DataLoader, TensorDataset

# Step 1: Define vocabulary
token_to_id = {
    "what": 0, "is": 1, "an": 2, "apple": 3, 
    "a": 4, "healthy": 5, "fruit": 6, "<EOS>": 7
}
id_to_token = {v: k for k, v in token_to_id.items()}

# Step 2: Read the text file
file_path = "data.txt"  # Replace with actual path

with open(file_path, "r") as f:
    sentences = [line.strip().lower().replace("?", "").split() for line in f]  # Tokenize words

# Step 3: Convert words to token IDs
inputs, labels = [], []

for sentence in sentences:
    token_ids = [token_to_id[word] for word in sentence if word in token_to_id]  # Convert to IDs

    if len(token_ids) > 0:
        inputs.append(token_ids)  # Inputs: Original sentence
        labels.append(token_ids[1:] + [token_to_id["<EOS>"]])  # Labels: Shifted sentence + <EOS>

# Step 4: Pad sequences for batch processing
max_length = max(len(seq) for seq in inputs)  # Find longest sequence

def pad_sequences(sequences, max_length, pad_value=7):
    return [seq + [pad_value] * (max_length - len(seq)) for seq in sequences]

inputs = torch.tensor(pad_sequences(inputs, max_length))
labels = torch.tensor(pad_sequences(labels, max_length))

# Step 5: Create a DataLoader
dataset = TensorDataset(inputs, labels)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

# Step 6: Print results
print("Inputs:\n", inputs)
print("Labels:\n", labels)


Inputs:
 tensor([[0, 1, 2, 7, 7],
        [2, 3, 1, 4, 5]])
Labels:
 tensor([[1, 2, 7, 7, 7],
        [3, 1, 4, 5, 7]])


In [16]:
num_tokens = len(token_to_id)
print(num_tokens)


8


In [17]:
model = DecoderOnlyTransformer(num_tokens=len(token_to_id), d_model=2, max_len=6)


## Now create the input for the transformer...
model_input = torch.tensor([token_to_id["what"], 
                            token_to_id["is"], 
                            token_to_id["an"],
                            token_to_id["apple"], 
                            token_to_id["<EOS>"]])
input_length = model_input.size(dim=0)

Seed set to 42


In [18]:
trainer = L.Trainer(max_epochs=300)
trainer.fit(model, train_dataloaders=dataloader)

You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type             | Params | Mode 
------------------------------------------------------------
0 | we             | Embedding        | 16     | train
1 | pe             | PositionEncoding | 0      | train
2 | self_attention | Attention        | 12     | train
3 | fc_layer       | Linear           | 24     | train
4 | loss           | CrossEntropyLoss | 0      | train
------------------------------------------------------------
52        Trainable params
0         Non-trainable params
52        Total params
0.000     Total estimated model params size (MB)
8         Modules in train mode
0         Modules in eval mode
C:\Users\Ltx\anaconda3\envs\dl_gpu\Lib\site-packages\lightning\

Training: |                                                                                                   …

`Trainer.fit` stopped: `max_epochs=300` reached.


In [19]:
model_input = torch.tensor([token_to_id["what"], 
                            token_to_id["is"], 
                            token_to_id["an"], 
                            token_to_id["apple"],
                            token_to_id["<EOS>"]])
input_length = model_input.size(dim=0)

predictions = model(model_input) 
predicted_id = torch.tensor([torch.argmax(predictions[-1,:])])
predicted_ids = predicted_id
max_length = 6
for i in range(input_length, max_length):
    if (predicted_id == token_to_id["<EOS>"]): # if the prediction is <EOS>, then we are done
        break
    
    model_input = torch.cat((model_input, predicted_id))
    predictions = model(model_input) 
    predicted_id = torch.tensor([torch.argmax(predictions[-1,:])])
    predicted_ids = torch.cat((predicted_ids, predicted_id))
        
print("Predicted Tokens:\n") 
for id in predicted_ids: 
    print("\t", id_to_token[id.item()])

Predicted Tokens:

	 <EOS>
